## **LLM의 한계를 너머, Tool & Agent**

### **Tavily AI로 검색 기반 에이전트 구축하기**

**Tavily AI를 인터넷 검색 Tool로 설정**

In [9]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

#Tavily AI API Key 환경변수 설정
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
os.environ['TAVILY_API_KEY']="YOUR_API_KEY"
tools = [TavilySearchResults()]

**Agent 구축**

In [10]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent

#Agent에서 Tool을 활용하기 위한 프롬프트
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
#LLM, Tool, 프롬프트를 사용하여 에이전트 생성
agent = create_openai_tools_agent(llm, tools, prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "GPT-4o가 뭐야?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'GPT-4o'}`


[{'url': 'https://openai.com/index/hello-gpt-4o/', 'content': 'Prior to GPT-4o, you could use Voice Mode to talk to ChatGPT with latencies of 2.8 seconds (GPT-3.5) and 5.4 seconds (GPT-4) on average. To achieve this, Voice Mode is a pipeline of three separate models: one simple model transcribes audio to text, GPT-3.5 or GPT-4 takes in text and outputs text, and a third simple model converts that text back to audio.'}, {'url': 'https://openai.com/index/gpt-4o-and-more-tools-to-chatgpt-free/', 'content': 'GPT-4o is our newest flagship model that provides GPT-4-level intelligence but is much faster and improves on its capabilities across text, voice, and vision. Today, GPT-4o is much better than any existing model at understanding and discussing the images you share. For example, you can now take a picture of a menu in a different language and talk to GPT-4o to translate it, learn

{'input': 'GPT-4o가 뭐야?',
 'output': 'GPT-4o는 OpenAI의 최신 주력 모델로, GPT-4 수준의 지능을 제공하지만 훨씬 빠르고 텍스트, 음성 및 이미지에 걸쳐 기능을 향상시켰습니다. GPT-4o는 텍스트 또는 이미지 입력에서 텍스트를 생성할 수 있는 다중 모달 모델입니다. 또한, GPT-4o는 이미지를 이해하고 토론하는 데 있어 기존 모델보다 훨씬 뛰어난 성능을 보여줍니다. 자세한 내용은 [여기](https://openai.com/index/hello-gpt-4o/)에서 확인할 수 있습니다.'}

### **Langchain으로 벡터 DB & 인터넷 검색 Agent 구축하기**

**Tavily AI를 인터넷 검색 Tool로 설정**

In [2]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

#Tavily AI API Key 환경변수 설정
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
os.environ['TAVILY_API_KEY']="YOUR_API_KEY"
search = TavilySearchResults()

**벡터 DB 구축**

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(r"C:\Users\gram\Downloads\★육아휴직제도 사용안내서_배포.pdf")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()


**create_retriever_tool로 벡터 DB 검색 툴 설정**

In [4]:
from langchain.tools.retriever import create_retriever_tool
#벡터 DB 검색을 위한 검색 도구 설정
retriever_tool = create_retriever_tool(
    retriever,
    "parental_leave",
    "육아휴직과 관련한 정보를 검색합니다. 육아 휴직 관련한 질문이 입력되면 이 도구를 사용합니다.",
)

**Agent 구축**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor

tools = [search, retriever_tool]
llm = ChatOpenAI(model="gpt-4o", temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "육아휴직은 어떻게 사용할 수 있어?"})



> Entering new AgentExecutor chain...

Invoking: `parental_leave` with `{'query': '육아휴직 사용 방법'}`




**벡터DB를 활용하는 질문**

In [ ]:
agent_executor.invoke({"input": "육아휴직은 어떻게 사용할 수 있어?"})

**인터넷 검색을 활용하는 질문**

In [ ]:
agent_executor.invoke({"input": "현재 대한민국의 인구는 몇명이야?"})